In [97]:
import numpy as np
import pandas as pd

In [101]:
url_path ='https://raw.githubusercontent.com/DZAlpha/ExoplanetHunting/main/exoTest.csv'
df = pd.read_csv(url_path, index_col=0)

# Setting index to double index : ID, Label
ID = np.arange(570)
label = df.index.values

multi_index = list(zip(ID,label))
index = pd.MultiIndex.from_tuples(multi_index, names=["ID", "LABEL"])
df = df.set_index(index)


In [99]:
df.head(100)

,,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,FLUX.40,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
ID,LABEL,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,119.88,100.21,86.46,48.68,46.12,39.39,18.57,6.98,6.63,-21.97,-23.17,-29.26,-33.99,-6.25,-28.12,-27.24,-32.28,-12.29,-16.57,-23.86,-5.69,9.24,35.52,81.20,116.49,133.99,148.97,174.15,187.77,215.30,246.80,-56.68,-56.68,-56.68,-52.05,-31.52,-31.15,-48.53,-38.93,-26.06,...,-2.55,12.26,-7.06,-23.53,2.54,30.21,38.87,-22.86,-22.86,-4.37,2.27,-16.27,-30.84,-7.21,-4.27,13.60,15.62,31.96,49.89,86.93,86.93,42.99,48.76,22.82,32.79,30.76,14.55,10.92,22.68,5.91,14.52,19.29,14.44,-1.62,13.33,45.50,31.93,35.78,269.43,57.72
1,2,5736.59,5699.98,5717.16,5692.73,5663.83,5631.16,5626.39,5569.47,5550.44,5458.80,5329.39,5191.38,5031.39,4769.89,4419.66,4218.92,3924.73,3605.30,3326.55,3021.20,2800.61,2474.48,2258.33,1951.69,1749.86,1585.38,1575.48,1568.41,1661.08,1977.33,2425.62,2889.61,3847.64,3847.64,3741.20,3453.47,3202.61,2923.73,2694.84,2474.22,...,-3470.75,-4510.72,-5013.41,-3636.05,-2324.27,-2688.55,-2813.66,-586.22,-586.22,-756.80,-1090.23,-1388.61,-1745.36,-2015.28,-2359.06,-2516.66,-2699.31,-2777.55,-2732.97,1167.39,1167.39,1368.89,1434.80,1360.75,1148.44,1117.67,714.86,419.02,57.06,-175.66,-581.91,-984.09,-1230.89,-1600.45,-1824.53,-2061.17,-2265.98,-2366.19,-2294.86,-2034.72
2,2,844.48,817.49,770.07,675.01,605.52,499.45,440.77,362.95,207.27,150.46,85.49,-20.12,-35.88,-65.59,-15.12,16.60,-25.70,61.88,53.18,64.32,72.38,100.35,67.26,14.71,-16.41,-147.46,-231.27,-320.29,-407.82,-450.48,-146.99,-146.99,-146.99,-146.99,-166.30,-139.90,-96.41,-23.49,13.59,67.59,...,-35.24,-70.13,-35.30,-56.48,-74.60,-115.18,-8.91,-37.59,-37.59,-37.43,-104.23,-101.45,-107.35,-109.82,-126.27,-170.32,-117.85,-32.30,-70.18,314.29,314.29,314.29,149.71,54.60,12.60,-133.68,-78.16,-52.30,-8.55,-19.73,17.82,-51.66,-48.29,-59.99,-82.10,-174.54,-95.23,-162.68,-36.79,30.63
3,2,-826.00,-827.31,-846.12,-836.03,-745.50,-784.69,-791.22,-746.50,-709.53,-679.56,-706.03,-720.56,-631.12,-659.16,-672.03,-665.06,-667.94,-660.84,-672.75,-644.91,-680.53,-620.50,-570.34,-530.00,-537.88,-578.38,-532.34,-532.38,-491.03,-485.03,-427.19,-380.84,-329.50,-286.91,-283.81,-298.19,-271.03,-268.50,-209.56,-180.44,...,16.50,-1286.59,-1286.59,-1286.59,-1286.59,-1286.59,-1286.59,-1286.59,-1286.59,-14.94,64.09,8.38,45.31,100.72,91.53,46.69,20.34,30.94,-36.81,-33.28,-69.62,-208.00,-280.28,-340.41,-337.41,-268.03,-245.00,-230.62,-129.59,-35.47,122.34,93.03,93.03,68.81,9.81,20.75,20.25,-120.81,-257.56,-215.41
4,2,-39.57,-15.88,-9.16,-6.37,-16.13,-24.05,-0.90,-45.20,-5.04,14.62,-19.52,-11.43,-49.80,25.84,11.62,3.18,-9.59,14.49,8.82,32.32,-28.90,-28.90,-14.09,-30.87,-18.99,-38.60,-27.79,9.65,29.60,7.88,42.87,27.59,27.05,20.26,29.48,9.71,22.84,25.99,-667.55,-1336.24,...,-122.12,-32.01,-47.15,-56.45,-41.71,-34.13,-43.12,-53.63,-53.63,-53.63,-24.29,22.29,25.18,1.84,-22.29,-26.43,-12.12,-33.05,-21.66,-228.32,-228.32,-228.32,-187.35,-166.23,-115.54,-50.18,-37.96,-22.37,-4.74,-35.82,-37.87,-61.85,-27.15,-21.18,-33.76,-85.34,-81.46,-61.98,-69.34,-17.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [102]:
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def standarize(series):
    return (series - np.mean(series)) / np.std(series)


normalized_df = df.apply(normalize,axis=1)
standarized_df = df.apply(standarize, axis=1)


print(normalized_df.head())
print(standarized_df.head())

normalized_df.head()

            FLUX.1    FLUX.2    FLUX.3  ...  FLUX.3195  FLUX.3196  FLUX.3197
ID LABEL                                ...                                 
0  2      0.656909  0.611783  0.580238  ...   0.463970   1.000000   0.514304
1  2      0.827188  0.824631  0.825831  ...   0.261207   0.266190   0.284360
2  2      1.000000  0.979432  0.943295  ...   0.232476   0.328413   0.379791
3  2      0.595693  0.595294  0.589562  ...   0.810594   0.768921   0.781765
4  2      0.843396  0.853817  0.856773  ...   0.833539   0.830301   0.852955

[5 rows x 3197 columns]
            FLUX.1    FLUX.2    FLUX.3  ...  FLUX.3195  FLUX.3196  FLUX.3197
ID LABEL                                ...                                 
0  2      3.996712  3.336906  2.875679  ...   1.175681   9.013181   1.911631
1  2      3.429783  3.407525  3.417970  ...  -1.496632  -1.453264  -1.295101
2  2      7.265724  7.035155  6.630059  ...  -1.338175  -0.262731   0.313220
3  2     -1.936815 -1.940136 -1.987824  ...  -0.149

,,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,FLUX.40,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
ID,LABEL,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,0.656909,0.611783,0.580238,0.493565,0.487692,0.472252,0.424488,0.397899,0.397096,0.331483,0.328730,0.314758,0.303907,0.367547,0.317374,0.319393,0.307830,0.353690,0.343871,0.327147,0.368832,0.403083,0.463374,0.568171,0.649132,0.689279,0.723646,0.781413,0.812659,0.875817,0.948083,0.251853,0.251853,0.251853,0.262474,0.309574,0.310422,0.270550,0.292574,0.322100,...,0.376035,0.410012,0.365689,0.327904,0.387712,0.451192,0.471059,0.329441,0.329441,0.371860,0.387093,0.344559,0.311134,0.365344,0.372089,0.413086,0.417720,0.455207,0.496341,0.581316,0.581316,0.480511,0.493748,0.434238,0.457111,0.452454,0.415265,0.406938,0.433917,0.395444,0.415196,0.426140,0.415013,0.378169,0.412466,0.486269,0.455138,0.463970,1.000000,0.514304
1,2,0.827188,0.824631,0.825831,0.824125,0.822106,0.819824,0.819491,0.815515,0.814186,0.807785,0.798745,0.789105,0.777930,0.759664,0.735200,0.721179,0.700629,0.678317,0.658846,0.637518,0.622109,0.599329,0.584231,0.562812,0.548714,0.537225,0.536534,0.536040,0.542513,0.564603,0.595916,0.628326,0.695245,0.695245,0.687810,0.667712,0.650189,0.630709,0.614721,0.599311,...,0.184053,0.111411,0.076298,0.172507,0.264135,0.238690,0.229951,0.385538,0.385538,0.373623,0.350333,0.329491,0.304572,0.285718,0.261705,0.250697,0.237939,0.232474,0.235587,0.508029,0.508029,0.522103,0.526707,0.521535,0.506705,0.504556,0.476419,0.455755,0.430472,0.414216,0.385840,0.357747,0.340508,0.314694,0.299042,0.282513,0.268207,0.261207,0.266190,0.284360
2,2,1.000000,0.979432,0.943295,0.870852,0.817896,0.737064,0.692346,0.633042,0.514403,0.471110,0.421599,0.341117,0.329106,0.306465,0.344927,0.369100,0.336864,0.403606,0.396976,0.405466,0.411608,0.432923,0.407706,0.367659,0.343944,0.244075,0.180206,0.112367,0.045663,0.013153,0.244433,0.244433,0.244433,0.244433,0.229718,0.249836,0.282978,0.338548,0.366806,0.407958,...,0.329594,0.303006,0.329548,0.313408,0.299599,0.268674,0.349659,0.327803,0.327803,0.327925,0.277019,0.279138,0.274641,0.272759,0.260223,0.226654,0.266640,0.331835,0.302967,0.595960,0.595960,0.595960,0.470538,0.398058,0.366051,0.254576,0.296886,0.316593,0.349934,0.341414,0.370029,0.317081,0.319649,0.310733,0.293884,0.223438,0.283878,0.232476,0.328413,0.379791
3,2,0.595693,0.595294,0.589562,0.592637,0.620225,0.608282,0.606292,0.619920,0.631187,0.640320,0.632253,0.627825,0.655081,0.646536,0.642614,0.644738,0.643861,0.646024,0.642395,0.650879,0.640024,0.658318,0.673604,0.685897,0.683496,0.671153,0.685184,0.685172,0.697773,0.699601,0.717227,0.731352,0.746998,0.759976,0.760921,0.756539,0.764816,0.765587,0.783548,0.792422,...,0.852438,0.455333,0.455333,0.455333,0.455333,0.455333,0.455333,0.455333,0.455333,0.842857,0.866941,0.849964,0.861218,0.878103,0.875303,0.861638,0.853608,0.856839,0.836192,0.837268,0.826194,0.784024,0.761997,0.743673,0.744587,0.765730,0.772748,0.777130,0.807918,0.836601,0.884692,0.875760,0.875760,0.868379,0.850399,0.853733,0.853581,0.810594,0.768921,0.781765
4,2,0.843396,0.853817,0.856773,0.858000,0.853707,0.850223,0.860406,0.840920,0.858585,0.867233,0.852216,0.855774,0.838896,0.872168,0.865913,0.862201,0.856584,0.867176,0.864682,0.875019,0.848090,0.848090,0.854604,0.847223,0.852449,0.843823,0.848578,0.865047,0.873822,0.864268,0.87965